In [1]:
import numpy as np

class TextDataloader:
    def __init__(self, dataset, max_seq_len, batch_size, shuffle=True):
        self.dataset = dataset
        self.max_seq_len = max_seq_len
        self.batch_size = batch_size
        self.dataset_len = len(dataset)
        
        # shuffle logic
        self.shuffle = shuffle
        self.chunk_len = max_seq_len * batch_size
        num_batches = math.ceil(self.dataset_len/self.chunk_len)
        self.batch_order = np.array(range(num_batches))
        
        if shuffle:
            np.random.shuffle(self.batch_order)

    def __iter__(self):
        self.index = 0
        return self

    def __next__(self):
        if self.index > len(self.batch_order) - 1:
            raise StopIteration

        i = self.batch_order[self.index]

        chunk_pos = i * self.chunk_len

        data = self.dataset[chunk_pos: chunk_pos + self.chunk_len]
        target = self.dataset[(chunk_pos) + 1: (chunk_pos + self.chunk_len) + 1]

        num_batches = min(self.batch_size, (self.dataset_len - chunk_pos) // self.max_seq_len)
        if num_batches == 0:
            raise StopIteration

        if(len(data) != len(target)):
            # remove mismatched batch sizes
            data = data.narrow(0, 0, self.max_seq_len * (num_batches))
            target = target.narrow(0, 0, self.max_seq_len * (num_batches))

        self.index += 1

        return self.batchify(data, target, num_batches)

    
    def batchify(self, data, target, num_batches):
        # Evenly divide the data across the batch_size batches.
        data = data.view(num_batches, -1).contiguous()
        target = target.view(num_batches, -1).contiguous()

        # shuffle data
        if self.shuffle:
            permutation = torch.randperm(data.size(0))
            data = data[permutation]
            target = target[permutation]
       
        # flatten targets
        target = target.reshape(-1)
        return data, target.reshape(-1)

In [2]:
import torch
import math
import numpy as np

length = 331
batch_size = 5
max_seq_len = 10

dataset = torch.arange(0, length)
dataloader = TextDataloader(dataset, max_seq_len, batch_size, False)
for batch in dataloader:
    data, targets = batch
    print("Data/Target Shapes", data.shape, targets.shape)
    print(data)
    print(targets)
    # break


Data/Target Shapes torch.Size([5, 10]) torch.Size([50])
tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
        [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
        [40, 41, 42, 43, 44, 45, 46, 47, 48, 49]])
tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
        37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])
Data/Target Shapes torch.Size([5, 10]) torch.Size([50])
tensor([[50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
        [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
        [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
        [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
        [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]])
tensor([ 51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
     

In [46]:
length = 300
batch_size = 5
seq_len = 10
chunk_len = seq_len * batch_size

num_batches = math.ceil(length/chunk_len)
num_batches
# t = torch.rand(num_batches)
# print('Original Tensor:', t)

order = np.array(range(5))
np.random.shuffle(order)
print('Order:', order)
# order[]

# in-place changing of values
data[np.array(range(5))] = data[order]
# print('New Tensor:', t)

# chunk_len = seq_len * batch_size
# print(chunk_len)
# data = torch.arange(0, length)
# data = batchify(data, batch_size)
# print(data.shape)

Order: [3 4 1 2 0]


In [52]:
data

5

In [58]:
r=torch.randperm(data.size(0))
d2 = data[r]
d2


tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
        [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
        [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
        [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]])